In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import plotly.express as px
import pandas as pd
import os
from AnimalShelter import AnimalShelter

# Initialize app
app = JupyterDash(__name__)

# MongoDB connection settings
username = "aacuser"
password = "Bocephus"
# Default to localhost:27017; change to nv-desktop-services.apporto.com:31580 if needed
mongo_host = "localhost"
mongo_port = 27017
# mongo_host = "nv-desktop-services.apporto.com"
# mongo_port = 31580

try:
    db = AnimalShelter(username, password, mongo_host, mongo_port)
    records = db.read({})  # Retrieve all records without limit
    df = pd.DataFrame.from_records(records)
except Exception as e:
    print(f"Error connecting to MongoDB or reading data: {e}")
    df = pd.DataFrame()  # Empty DataFrame as fallback

# Remove '_id' column
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Map latitude/longitude fields
if 'location_lat' not in df.columns and 'latitude' in df.columns:
    df['location_lat'] = df['latitude']
if 'location_long' not in df.columns and 'longitude' in df.columns:
    df['location_long'] = df['longitude']

# Grazioso Salvare logo
logo_path = os.path.join('assets', 'grazioso_logo.png')
if os.path.exists(logo_path):
    logo_component = html.A(
        html.Img(src='/assets/grazioso_logo.png', style={'height': '80px', 'margin': '10px'}),
        href='https://www.snhu.edu', target='_blank'
    )
else:
    logo_component = html.Div(
        "Error: grazioso_logo.png not found in assets/ folder",
        style={'color': 'red', 'margin': '10px'}
    )

# Rescue options
RESCUE_OPTIONS = [
    {"label": "Water Rescue", "value": "Water Rescue"},
    {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
    {"label": "Disaster or Individual Tracking", "value": "Disaster or Individual Tracking"},
    {"label": "Reset", "value": "Reset"}
]

# Rescue queries
RESCUE_QUERIES = {
    "Water Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Mountain or Wilderness Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Disaster or Individual Tracking": {
        "animal_type": "Dog",
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    },
    "Reset": {}
}

# Helper function to run queries
def run_query_and_prepare(query):
    try:
        recs = db.read(query)  # No limit applied
        if not recs:
            return []
        dlocal = pd.DataFrame.from_records(recs)
        if '_id' in dlocal.columns:
            dlocal.drop(columns=['_id'], inplace=True)
        if 'location_lat' not in dlocal.columns and 'latitude' in dlocal.columns:
            dlocal['location_lat'] = dlocal['latitude']
        if 'location_long' not in dlocal.columns and 'longitude' in dlocal.columns:
            dlocal['location_long'] = dlocal['longitude']
        return dlocal.to_dict('records')
    except Exception as e:
        print(f"Error in query: {e}")
        return []

# Dashboard layout
app.layout = html.Div([
    html.Div([
        logo_component,
        html.H1('CS-340 Dashboard - Corey Arnold', style={'marginLeft': '20px', 'display': 'inline-block', 'verticalAlign': 'middle'})
    ], style={'display': 'flex', 'alignItems': 'center', 'gap': '20px'}),
    html.Hr(),
    html.Div([
        html.Label("Choose a rescue filter:", style={'fontWeight': 'bold'}),
        dcc.RadioItems(
            id='filter-type',
            options=RESCUE_OPTIONS,
            value='Reset',
            labelStyle={'display': 'block', 'margin': '10px'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns] if not df.empty else [],
        data=df.to_dict('records') if not df.empty else [],
        page_size=10,
        filter_action="native",
        sort_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto', 'maxHeight': '400px', 'overflowY': 'auto'},
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px',
            'maxWidth': '200px',
            'whiteSpace': 'normal',
            'padding': '5px'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ]
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6', style={'width': '50%'}),
            html.Div(id='map-id', className='col s12 m6', style={'width': '50%'})
        ]
    )
])

# Callback for updating all widgets
@app.callback(
    [
        Output('datatable-id', 'data'),
        Output('graph-id', 'children'),
        Output('map-id', 'children')
    ],
    [Input('filter-type', 'value'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_dashboard(filter_value, selected_rows):
    try:
        query = RESCUE_QUERIES.get(filter_value, {})
        data = run_query_and_prepare(query)
        
        table_data = data
        
        dff = pd.DataFrame.from_records(data) if data else pd.DataFrame()
        if dff.shape[0] == 0 or 'breed' not in dff.columns:
            pie_chart = html.Div("No data to chart.")
        else:
            counts = dff['breed'].value_counts().reset_index()
            counts.columns = ['breed', 'count']
            fig = px.pie(
                counts.head(15),
                names='breed',
                values='count',
                title='Top Breeds in View',
                color_discrete_sequence=px.colors.qualitative.Set3
            )
            pie_chart = [dcc.Graph(figure=fig)]
        
        if not data or selected_rows is None or not dff.shape[0]:
            map_chart = [dl.Map(
                style={'width': '1000px', 'height': '500px'},
                center=[30.75, -97.48],
                zoom=10,
                children=[dl.TileLayer(id="base-layer-id")]
            )]
        else:
            dff = pd.DataFrame.from_records(data)
            row = selected_rows[0] if selected_rows else 0
            # Ensure location_lat and location_long exist
            if 'location_lat' in dff.columns and 'location_long' in dff.columns:
                lat = dff.iloc[row]['location_lat']
                lon = dff.iloc[row]['location_long']
                breed = dff.iloc[row]['breed'] if 'breed' in dff.columns else "Unknown"
                name = dff.iloc[row]['name'] if 'name' in dff.columns else "Unknown"
                map_chart = [
                    dl.Map(
                        style={'width': '1000px', 'height': '500px'},
                        center=[30.75, -97.48],
                        zoom=10,
                        children=[
                            dl.TileLayer(id="base-layer-id"),
                            dl.Marker(
                                position=[lat, lon],
                                children=[
                                    dl.Tooltip(breed),
                                    dl.Popup([
                                        html.H1("Animal Name"),
                                        html.P(name)
                                    ])
                                ]
                            )
                        ]
                    )
                ]
            else:
                map_chart = [dl.Map(
                    style={'width': '1000px', 'height': '500px'},
                    center=[30.75, -97.48],
                    zoom=10,
                    children=[dl.TileLayer(id="base-layer-id")]
                )]
        
        return table_data, pie_chart, map_chart
    except Exception as e:
        print(f"Error in update_dashboard: {e}")
        return [], [html.Div("Error generating chart.")], [dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[dl.TileLayer(id="base-layer-id")]
        )]

# Highlight selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Run server
app.run_server(debug=True)

Error connecting to MongoDB or reading data: __init__() takes from 1 to 3 positional arguments but 5 were given
Dash app running on http://127.0.0.1:11285/
Read operation error: nv-desktop-services.apporto.com:31580: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 68ab81322151f57809eb9dab, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 31580) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:31580: [Errno 111] Connection refused')>]>
Read operation error: nv-desktop-services.apporto.com:31580: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 68ab81322151f57809eb9dab, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 31580) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:31580: [Errno 111] Connection refused')>]>
Read operation error: nv-desktop-ser